# Backpropagation

Any gradient-based optimization method needs the gradient. Moreover, if the gradient is available, then gradient-based methods tend to be fast. The fact that it is indeed possible to calculate gradients efficiently probably goes a long way towards explaining the current popularity of gradient-based methods for training neural networks. This algorithm, known as backpropagation, is the topic of this notebook.

There is nothing particularly deep about backpropagation. In essence it is just an efficient way of calculating the chain rule (calculus). The basic ideas are easily explained by means of a simple example.

---


## Simple example

<figure> 
<figcaption>**The forward pass**</figcaption>
<img src="./images/backprop_3.png"
     width=500px
     alt="The forward pass"
     style="float: center"/> 
</figure>

<p>In the left-most figure the computational graph of the function $f(x,y) = x^2(xy+1)$ is shown. Note that we also label the computational nodes from $n_1$ to $n_7$, where $n_7$ is simply the value of $f$. The graph shows the flow from the input nodes $n_1=x,\;\;n_2=y$, and $n_3=1$, through the graph up to the output node $n_7$ which is just $f$, as said above. You can think of this as a symbolic graph illustrating the connections between the different nodes. The Figure on the right shows how actual input data $x=2,\; y=1$, and $n_3=1$ flows through the graph. The values of all the nodes are saved for future use, more specifically, during the backward pass.</p>

<p>The goal is to calculate the gradient $\nabla f(x,y)=\left[ \begin{array}{c} \frac{\partial f}{\partial x}\\ \frac{\partial f}{\partial y}\end{array} \right]$. For this we clearly need to calculate the two partial derivatives. If we do it ourselves, we'll simply apply the chain rule to obtain the answers. Backpropagation does this in a systematic manner.  </p>

<figure> 
<figcaption>**The backward pass**</figcaption>
<img src="./images/backprop_2.png"
     width=500px
     alt="The backward pass"
     style="float: center"/> 
</figure>


<p>Since the partial derivatives of $f$ with respect to $x$ and $y$ are required, we proceed from the top. Since $f = n_7$ it follows that $\frac{\partial f}{\partial n_7} = 1$. From $n_7$ both branches should be followed, as indicated in the left-most Figure on the right. The first thing to note is that all the nodal values that we require to calculate are already available from the forward pass. It is therefore straightforward to calculate all the partial derivatives with respect to all the nodes, systematically, starting at the top, output, node. Working our way 'backwards' through the graph, we eventually arrive at $x$, via two different paths. The partial derivative with respect to $x$ is simply the sum of the partial derivatives along the two paths. Since $y$ is reached from only one path, the partial derivative as calculated along that path, is the required partial derivative.</p>

----

## Fully connected neural network

<figure> 
<figcaption>**Connection between two layers**</figcaption>
<img src="./images/nn_backprop.png"
     width=500px
     alt="Connected layers"
     style="float: center"/> 
</figure>

In order to illustrate how this can be made into a systematic algorithm, we'll now consider a fully connected neural network with activation functions $\sigma(\cdot)$. Keep in mind that $\sigma$ is a general activation function, not necessarily the sigmoid function, and that it can even be a different function for each layer. All that is important is that we'll be able to calculate its derivative $\sigma^{\prime}(\cdot)$. It may be worth pointing out that $\sigma$ is applied pointwise on vectors. If $\mathbf{x}=\left[ \begin{array}{c} x_1\\ \vdots \\ x_d \end{array}\right]$, then  $\sigma(\mathbf{x})$ is a vector with components $\sigma(\mathbf{x})=\left[ \begin{array}{c} \sigma(x_1)\\ \vdots \\ \sigma(x_d) \end{array}\right]$.

The Figure on the left shows how information from layer $\ell-1$ is passed to layer $\ell$. The output of the top layer, given an input vector $\mathbf{x}$, is a vector $\mathbf{a}^L(\mathbf{x})$. The mean cost function is then given by the average over all input vectors,
$$ C(W) = \frac1N \sum_{\mathbf{x}} C_{\mathbf{x}}(W),$$ 
where $C_{\mathbf{x}}(W)$ is the cost function for a single input vector, and $W$ indicates all the network parameters. Thus, $C_{\mathbf{x}}(W)$ is a function of the model output $\mathbf{a}^L(\mathbf{x})$ and the targets $\mathbf{t}$, not indicated in the notation.

**Notes:**

* The expression of the average cost function is of course an assumption that it can be written as the mean over individual input vectors.
* Examples of often-used cost functions of this form include mse and cross-entropy.

<p>We need to calculate the gradient of $C(W)$ with respect to all the parameters. Note that this is just the sum of the gradients for the individual input vectors $C_{\mathbf{x}}(W)$. More specifically, we need to calculate the gradients with respect to all the offsets,
$\frac{\partial C}{\partial b_k^{\ell}}$ and all the weights $\frac{\partial C}{\partial w_{jk}^{\ell}}$, where we have now dropped the dependencies on $\mathbf{x}$ and $W$ in the notation. Also note that $w_{jk}^{\ell}$ is the weight connecting $k$-th neuron in layer $\ell-1$ to the $j$-th neuron in the  $\ell$-th layer, as shown in the Figure. </p>

<p> Referring to the Figure above, we can therefore write

\begin{align}
\frac{\partial C}{\partial w_{jk}^{\ell}} &= \frac{\partial C}{\partial z_j^{\ell}} \frac{\partial z_j^{\ell}}{\partial w_{jk}^{\ell}}\\
&= \delta_j^{\ell} a_k^{\ell-1}, \end{align}
where we somewhat ambiguously refer to $\delta_j^{\ell}=\frac{\partial C}{\partial z_j^{\ell}}$ as the 'error'. The partial derivatives with respect to the offsets are given by,
\begin{align}
\frac{\partial C}{\partial b_{j}^{\ell}} &= \frac{\partial C}{\partial z_j^{\ell}} \frac{\partial z_j^{\ell}}{\partial b_{j}^{\ell}}\\
&= \delta_j^{\ell}. \end{align}</p>


**Note:**<br>

* The values of the activations $a_k^{\ell}$ are calculated during the forward sweep and stored in memory. Assuming of course that enough memory is available.


<figure> 
<figcaption>**Connection between layer $\ell+1$ and layer $\ell$**</figcaption>
<img src="./images/nn_backprop_2.png"
     width=500px
     alt="Backward connected layers."
     style="float: left"/> 
</figure>
<p>Let's see how the errors propagate through the network, starting from the output layer,
\begin{align}
\delta_j^L &= \frac{\partial C}{\partial z_j^L}\\
&= \frac{\partial C}{\partial a_j^L}\frac{\partial a_j^L}{\partial z_j^L}\\
&= \frac{\partial C}{\partial a_j^L} \sigma^{\prime}(z_j^L).
\end{align}
If $\delta_j^{\ell}$ can be calculated from the $\delta_i^{\ell+1}$ of the upper layer, we have a recursion, and an algorithm for calculating the gradients.</p>



<p>We start with,
\begin{align}
\delta_j^{\ell} &= \frac{\partial C}{\partial z_j^{\ell}}\\
&= \sum_i \frac{\partial C}{\partial z_i^{\ell+1}}\frac{\partial z_i^{\ell+1} }{\partial z_j^{\ell}}\\
&= \sum_i\delta_i^{\ell+1} \frac{\partial z_i^{\ell+1} }{\partial z_j^{\ell}}.
\end{align}
Since
\begin{align}
    z_i^{\ell+1} &= \sum_k w_{ik}^{\ell+1} a_k^{\ell} + b_i^{\ell+1}\\
    &= \sum_k w_{ik}^{\ell+1} \sigma(z_k^{\ell}) + b_i^{\ell+1},
\end{align}
it follows that
$$
\frac{\partial z_i^{\ell+1}}{\partial z_j^{\ell}} = w_{ij}^{\ell+1} \sigma^{\prime}(z_j^{\ell}),
$$
and
$$
\delta_j^{\ell} = \sum_i \delta_i^{\ell+1} w_{ij}^{\ell+1} \sigma^{\prime}(z_j^{\ell}).
$$
    </p>
    
<p> This allows a recursion. Starting from the top, we calculate the $\delta$'s through the lower layers, all the way down to the bottom layer. Since we have all the necessary values in memory from the forward sweep, this is fast. Moreover, from the $\delta$'s we calculate all the necesary partial derivatives.</p>
<p> The Figure below gives a schemetic overview of how the different layers fit together.</p>
<br><br>
    
    <figure> 
<figcaption>**How the layers fit together**</figcaption>
<img src="./images/full_nn_backprop.png"
     width=800px
     alt="Fit together"
     style="float: center"/> 
</figure>
We are now in a position to indicate why it is hard to train deep networks. First note that the update in a parameter is proportional to its component in the gradient. To be more specific the update in $w_{jk}^{\ell}$ is proportional to $\frac{\partial C}{\partial w_{jk}^{\ell}}$, given by,
$$
\frac{\partial C}{\partial w_{jk}^{\ell}} = \delta_j^{\ell} a_{k}^{\ell-1}.
$$
<p>If this quantity is small, the update is small and it may take a long time to train the parameter. Thus, if either $a_{k}^{\ell-1}$ or $\delta_j^{\ell}$ is small, training will slow down. The latter depends on the derivative of the activation function, $\sigma^{\prime}(z_j^{\ell})$. The sigmoid activation function quickly saturates as the logits move away from 0, slowing down the training. Moreover, the values in the different layers are essentially multiplied, meaning that the gradient can become very small just before the input layer. But this is exactly where the information is fed into the network. </p>

<p>On the other hand, if the derivative of the activation function is larger than one, then this value is multiplied as backpropagation passes through the different layers. This can lead to exploding gradients. </p>

<p>It is clearly important to avoid both vanishing, and exploding gradients through a judicious choice of the the activation function, among other considerations.</p>

